In [60]:
import pandas as pd
import numpy as np  
from pathlib import Path
import os

In [61]:
s3_path = (
    "s3a://pop-eu-prod/390/200_feed/0/0/4e/056920.txt" 
)

stock_feed_df = pd.read_csv(s3_path, sep="\t")
stock_feed_df

,SKU,Stock,Price,WholeSale Price,Sin
0,720-021,In Stock,99.99 GBP,73.99 GBP,1O366AQUOA000
1,01-0710,Out of Stock,99.99 GBP,74.99 GBP,15K1T542QF801
2,841-094,In Stock,179.99 GBP,119.99 GBP,15K2C8K9AF801
3,AA1-069RD,In Stock,59.99 GBP,44.99 GBP,15KFM0J8AF001
4,835-388WT,In Stock,69.99 GBP,45.99 GBP,15KGDBUMIF001
...,...,...,...,...,...
10955,833-695V72DB,7,179.99 GBP,164.99 GBP,21D6BGADHKO01
10956,833-695V74YL,In Stock,179.99 GBP,137.99 GBP,21D6BGAS1TO00
10957,833-695V72LG,In Stock,459.99 GBP,136.99 GBP,21DQV4CKHNO00
10958,833-695V74BG,Out of Stock,179.99 GBP,154.99 GBP,21FOO8TFPL801


In [62]:
folder_path = "order history data/"
all_dataframes = []

# 2. Loop through every file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(folder_path, filename)
        
        try:
            # sheet_name=1 targets the second sheet
            df = pd.read_excel(file_path, sheet_name='SKU', skiprows=2)
            df = df.iloc[:, 0:3]
            # consistent naming
            df.columns = ['SKU', 'Quantity', 'Sales Amount']

            df['source_file'] = " ".join(filename.split(" ")[1:3])
            df['date'] = pd.to_datetime(df['source_file'], format='%B %Y')
            
            all_dataframes.append(df)
            print(f"Successfully read: {filename}")

        except Exception as e:
            print(f"Could not read {filename}: {e}")

# 3. Combine all sheets into one large DataFrame
if all_dataframes:
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    print("All files merged successfully!")
else:
    print("No Excel files found.")

combined_df = combined_df[combined_df['SKU'] != 'Grand Total'] #getting rid of the Grand Total row

Successfully read: 02. February 2024 SKU sales per platform.xlsx
Successfully read: 01. January 2024 SKU sales per platform.xlsx
Successfully read: 02. February 2025 SKU sales per platform.xlsx
Successfully read: 01. January 2025 SKU sales per platform.xlsx
Successfully read: 03. March 2024 SKU sales per platform.xlsx
All files merged successfully!


In [63]:
combined_df

,SKU,Quantity,Sales Amount,source_file,date
0,845-953V00GY,40,12778.58,February 2024,2024-02-01
1,845-030V00LG,55,10820.28,February 2024,2024-02-01
2,B20-056,79,9208.32,February 2024,2024-02-01
3,845-059V01,23,9077.97,February 2024,2024-02-01
4,E2-0007,73,8281.97,February 2024,2024-02-01
...,...,...,...,...,...
28062,U-845-386V01,1,8.99,March 2024,2024-03-01
28063,U-845-386,1,8.99,March 2024,2024-03-01
28064,U-84A-309V00CG,1,8.99,March 2024,2024-03-01
28065,U-D30-466,1,7.99,March 2024,2024-03-01


In [64]:
winter_2024_df = combined_df[(combined_df['date']>=pd.to_datetime('2024-01-01')) & (combined_df['date']<=pd.to_datetime('2024-02-28'))]
winter_2024_df = winter_2024_df.groupby('SKU').agg({'Quantity':'sum', 'Sales Amount':'sum'}).reset_index()
winter_2024_df = winter_2024_df.sort_values(by='Quantity', ascending=False)
winter_2024_df = winter_2024_df.merge(stock_feed_df[['SKU', 'Stock', 'WholeSale Price']], on='SKU', how='left')

winter_2025_df = combined_df[(combined_df['date']>=pd.to_datetime('2025-01-01')) & (combined_df['date']<=pd.to_datetime('2025-02-28'))]
winter_2025_df = winter_2025_df.groupby('SKU').agg({'Quantity':'sum', 'Sales Amount':'sum'}).reset_index()
winter_2025_df = winter_2025_df.sort_values(by='Quantity', ascending=False)
winter_2025_df = winter_2025_df.merge(stock_feed_df[['SKU', 'Stock', 'WholeSale Price']], on='SKU', how='left')

comparison_df = winter_2024_df.merge(
    winter_2025_df, 
    on='SKU', 
    how='inner', 
    suffixes=('_2024', '_2025')
)

# Create a 'Total Quantity' column to find the overall most popular items
comparison_df['Total_Quantity'] = comparison_df['Quantity_2024'] + comparison_df['Quantity_2025']

# Sort by the highest total volume
popular_common_products = comparison_df.sort_values(by='Total_Quantity', ascending=False)


In [67]:
popular_common_products.head(20)

,SKU,Quantity_2024,Sales Amount_2024,Stock_2024,WholeSale Price_2024,Quantity_2025,Sales Amount_2025,Stock_2025,WholeSale Price_2025,Total_Quantity
873,821-022V70WT,13,1653.07,In Stock,87.99 GBP,282,22431.12,In Stock,87.99 GBP,295
10,821-019V70,119,17792.98,In Stock,109.99 GBP,164,16139.32,In Stock,109.99 GBP,283
8,A91-020V01,135,4092.67,In Stock,24.99 GBP,112,2889.43,In Stock,24.99 GBP,247
2,E30-001,166,2826.89,29,19.99 GBP,78,1469.63,29,19.99 GBP,244
1,820-265V70WT,171,7533.26,17,59.99 GBP,67,2531.97,17,59.99 GBP,238
0,837-067V70ND,193,11076.40,In Stock,54.99 GBP,20,1432.25,In Stock,54.99 GBP,213
152,01-0482,43,1767.52,In Stock,34.99 GBP,166,5685.64,In Stock,34.99 GBP,209
13,A91-062,108,6906.40,In Stock,63.99 GBP,95,5731.16,In Stock,63.99 GBP,203
33,800-088,77,2574.46,In Stock,26.99 GBP,125,3400.31,In Stock,26.99 GBP,202
29,A91-023V01,83,3831.77,In Stock,39.99 GBP,109,4266.64,In Stock,39.99 GBP,192
